In [1]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [2]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data,procedure_data,procedure_data_param
from abastecimientov2 import reservas_avisorden_ordenes
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [3]:
consulta = "select fechacarga,identificador,orden,reserva_centro,ate_accion,estado_compra,ctd_dif,material from tbl_abastecimiento_hist"

In [4]:
connection = crear_conexion()
necesidades_mayo=consult_data(connection,consulta)
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [5]:
consulta2 = "select fechacarga,orden,reserva,reservapos,reserva_centro,ate_accion,estado_compra,ctd_dif,material from p_planordenreserva_hist where fechacarga between '2024-12-01' and '2025-02-28'"

In [6]:
connection = crear_conexion()
necesidaddes_enero=consult_data(connection,consulta2)
cerrar_conexion(connection)

Conexion Exitosa
conexion cerrada


In [7]:
df_necesidades_enero = pd.DataFrame(necesidades_mayo)
df_necesidades_mayo = pd.DataFrame(necesidaddes_enero)

In [8]:
df_necesidades_enero_ = df_necesidades_enero.copy()

In [9]:
df_necesidaddes_mayo_ = df_necesidades_mayo.copy()

In [10]:
df_necesidaddes_mayo_['identificador'] = (df_necesidaddes_mayo_['orden'].fillna('')+df_necesidaddes_mayo_['reserva']+df_necesidaddes_mayo_['reservapos'])

In [11]:
df_necesidaddes_mayo_columnas = df_necesidaddes_mayo_[['fechacarga','identificador','orden','reserva_centro','ate_accion','estado_compra','ctd_dif']].copy()

In [12]:
df_necesidades_enero_columnas = df_necesidades_enero_[['fechacarga','identificador','orden','reserva_centro','ate_accion','estado_compra','ctd_dif']].copy()

In [13]:
df_necesidades_ = pd.concat([df_necesidades_enero_columnas, df_necesidaddes_mayo_columnas], ignore_index=True)

In [14]:
df_necesidades_validas = df_necesidades_.loc[(df_necesidades_['ate_accion'] == 'MERCADERIA') & (df_necesidades_['ctd_dif'] > 0)].copy()

In [15]:
df_necesidades_validas['fecha'] = df_necesidades_validas['fechacarga'].dt.date

In [16]:
df_necesidades_validas_solo_una_fecha = df_necesidades_validas.drop_duplicates(subset=['fecha','identificador'],keep='first')

In [17]:
df_necesidades_validas_solo_una_fecha['grupo'] = ''
df_necesidades_validas_solo_una_fecha.loc[df_necesidades_validas_solo_una_fecha['orden'].str[0] =='P','grupo'] = 'Personal'
df_necesidades_validas_solo_una_fecha.loc[df_necesidades_validas_solo_una_fecha['orden'].str[0] =='4','grupo'] = 'Equipo'

C:\Users\CGM\AppData\Local\Temp\ipykernel_12196\4253203555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_necesidades_validas_solo_una_fecha['grupo'] = ''


In [18]:
df_necesidades_validas_ = df_necesidades_validas_solo_una_fecha.loc[(df_necesidades_validas_solo_una_fecha['grupo'] != '')]

In [19]:
df_necesidades_validas_fil = df_necesidades_validas_[['fecha','identificador','reserva_centro','estado_compra','grupo']].copy()

In [20]:
def agregar_dias_consecutivos_estado(df_necesidades):
    """
    Agrega una columna con el conteo de días consecutivos en el mismo estado hacia atrás,
    reiniciando el contador si hay un salto de más de 7 días entre fechas o si el estado cambia.

    Parámetros:
    - df_necesidades: DataFrame con columnas ['fecha', 'identificador', 'estado_compra', ...]

    Retorna:
    - DataFrame con columna 'dias_consecutivos_estado' agregada
    """

    df_necesidades['fecha'] = pd.to_datetime(df_necesidades['fecha'])
    df_necesidades = df_necesidades.sort_values(['identificador', 'fecha']).reset_index(drop=True)

    # Comparar con la fila anterior por identificador
    df_necesidades['estado_anterior'] = df_necesidades.groupby('identificador')['estado_compra'].shift(1)
    df_necesidades['fecha_anterior'] = df_necesidades.groupby('identificador')['fecha'].shift(1)

    # Calcular diferencia de días
    df_necesidades['diferencia_dias'] = (df_necesidades['fecha'] - df_necesidades['fecha_anterior']).dt.days

    # Inicializar columna
    df_necesidades['dias_consecutivos_estado'] = 1

    # Corrección final: recorrer por grupo para aplicar la lógica completa
    for identificador in df_necesidades['identificador'].unique():
        sub_df = df_necesidades[df_necesidades['identificador'] == identificador]
        contador = 0
        estado_prev = None
        fecha_prev = None

        for idx, row in sub_df.iterrows():
            if estado_prev is None or row['estado_compra'] != estado_prev or (fecha_prev is not None and (row['fecha'] - fecha_prev).days > 7):
                contador = 1
            else:
                contador += 1
            df_necesidades.at[idx, 'dias_consecutivos_estado'] = contador
            estado_prev = row['estado_compra']
            fecha_prev = row['fecha']

    # Limpiar columnas auxiliares
    return df_necesidades.drop(columns=['estado_anterior', 'fecha_anterior', 'diferencia_dias'])


In [21]:
df_resultado1 = agregar_dias_consecutivos_estado(df_necesidades_validas_fil)

In [22]:
df_abastecimiento = df_resultado1.copy()

In [23]:
df_abastecimiento['necesidad'] = 1
df_abastecimiento['abastecido'] = 0
df_abastecimiento.loc[df_abastecimiento['estado_compra'].isin(['Total_abs_inv','Segmien_Oc']) , 'abastecido'] = 1

In [24]:
df_abastecimiento['rango_dias'] = ''
def asignar_rango_dias(row):
    if row['dias_consecutivos_estado'] <= 1:
        return '0-1'
    if row['dias_consecutivos_estado'] <= 3:
        return '2-3'
    elif row['dias_consecutivos_estado'] <= 7:
        return '4-7'
    else:
        return '7+'

df_abastecimiento['rango_dias'] = df_abastecimiento.apply(asignar_rango_dias, axis=1)

In [25]:
df_abastecimiento_columns = df_abastecimiento[['fecha', 'reserva_centro', 'grupo', 'rango_dias', 'necesidad', 'abastecido']].copy()

In [26]:
df_abastecimiento.to_excel('abastecimiento_2025.xlsx', index=False)

In [27]:
df_abastecimiento_agrupado = df_abastecimiento_columns.groupby(['fecha', 'reserva_centro', 'grupo', 'rango_dias']).agg({'necesidad': 'sum', 'abastecido': 'sum'})

In [28]:
df_abastecimiento_agrupado.reset_index(inplace=True)

In [29]:
df_abastecimiento_agrupado['abastecimiento'] = df_abastecimiento_agrupado['abastecido'] / df_abastecimiento_agrupado['necesidad']

In [30]:
df_abastecimiento_agrupado['Mes'] = df_abastecimiento_agrupado['fecha'].dt.month

In [31]:
df_abastecimiento_agrupado.to_excel('abastecimiento_agrupado.xlsx', index=False)

In [32]:
# Asumiendo que tu DataFrame se llama df
tabla = pd.pivot_table(
    df_abastecimiento_agrupado,
    index=['Mes', 'reserva_centro'],
    columns=['rango_dias', 'grupo'],
    values=['abastecimiento', 'necesidad'],
    aggfunc='mean'
)

# Opcional: para ordenar bien las columnas y que se vea más limpio
tabla = tabla.sort_index(axis=1, level=[0,1])

tabla.columns = ['_'.join(col).strip() for col in tabla.columns.values]


In [33]:
tabla = tabla.round(2)

In [34]:
tabla.to_excel('tabla_abastecimiento.xlsx', index=True)

In [35]:
tabla

abastecimiento_0-1_Equipo  abastecimiento_0-1_Personal  \
Mes reserva_centro                                                           
1   C040                                 0.67                         0.78   
    C080                                 0.67                         0.69   
    C154                                 0.73                         0.74   
    C200                                 0.59                         0.53   
2   C040                                 0.83                         0.83   
    C080                                 0.74                         0.65   
    C152                                 1.00                          NaN   
    C154                                 0.80                         0.79   
    C200                                 0.75                         0.61   
3   C040                                 0.81                         0.81   
    C080                                 0.81                         0.74   
    C152                                 1.00                          NaN   
    C154                                 0.80                         0.74   
    C200                                 0.73                         0.63   
4   C040                                 0.83                         0.73   
    C080                                 0.78                         0.65   
    C154                                 0.78                         0.69   
    C200                                 0.84                         0.27   
5   C040                                 0.80                         0.87   
    C080                                 0.82                         0.72   
    C154                                 0.80                         0.86   
    C200                                 0.83                         0.56   
6   C040                                 0.83                         0.88   
    C080                                 0.77                         0.56   
    C152                                 1.00                          NaN   
    C154                                 0.84                         0.87   
    C200                                 0.83                         0.82   
7   C040                                 0.96                         0.79   
    C080                                 0.78                          NaN   
    C154                                 0.96                         0.78   
    C200                                 0.88                         1.00   
12  C040                                 0.67                         0.75   
    C080                                 0.66                         0.58   
    C152                                 0.00                          NaN   
    C154                                 0.75                         0.68   
    C200                                 0.55                         0.43   

                    abastecimiento_2-3_Equipo  abastecimiento_2-3_Personal  \
Mes reserva_centro                                                           
1   C040                                 0.70                         0.77   
    C080                                 0.71                         0.46   
    C154                                 0.76                         0.65   
    C200                                 0.61                         0.57   
2   C040                                 0.88                         0.93   
    C080                                 0.86                         0.76   
    C152                                 1.00                          NaN   
    C154                                 0.88                         0.83   
    C200                                 0.84                         0.59   
3   C040                                 0.87                         0.73   
    C080                                 0.80                         0.78   
    C152                                 